In [2]:
from python_graphql_client import GraphqlClient
import datetime

# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

#timestamp = "2021-08-25T15:52:43.785+02:00"
timestamp = datetime.datetime.now().astimezone().isoformat()

# Create the query string and variables required for the request.
query = """
query t($fromPlace: String, $toPlace: String, $timestamp: DateTime){
  trip(
    from: {place: $fromPlace}
    to: {place: $toPlace}
    numTripPatterns: 3
    dateTime: $timestamp
    walkSpeed: 1.3
    arriveBy: false
  ) {
    tripPatterns {
      expectedStartTime
      legs {
        mode
        toPlace {
          name
        }
        interchangeTo {
          guaranteed
          ToServiceJourney {
            privateCode
          }
        }
        interchangeFrom {
          guaranteed
          ToServiceJourney {
            id
          }
        }
      }
    }
 }
}
"""


variables = {
    "fromPlace": "NSR:Quay:99503",
    "toPlace": "NSR:Quay:72050",
    "timestamp": timestamp,
}

data = client.execute(query=query, variables=variables)
data

{'data': {'trip': {'tripPatterns': [{'expectedStartTime': '2021-08-24T13:38:47+02:00',
     'legs': [{'mode': 'foot',
       'toPlace': {'name': 'Trondheim lufthavn'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Buran'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Siemens'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'foot',
       'toPlace': {'name': 'Siemens'},
       'interchangeTo': None,
       'interchangeFrom': None}]},
    {'expectedStartTime': '2021-08-24T13:44:05+02:00',
     'legs': [{'mode': 'foot',
       'toPlace': {'name': 'Sandfærhus'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Stjørdal stasjon'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Siemens'},
       'int

In [2]:
for i in data['data']:
    trippattern = data['data'][i]['tripPatterns']
    for pattern in trippattern:
        print(pattern['expectedStartTime'])
        legs = pattern['legs']
        for leg in legs:
            # {'mode': 'bus', 'toPlace': {'name': 'Dronningens gate'}, 'interchangeTo': {'guaranteed': True, 'ToServiceJourney': {'privateCode': '65'}}, 'interchangeFrom': None}

            #print(leg)
            t = f'\tleg {leg["mode"]} to "{leg["toPlace"]["name"]}"'
            
            interchangeTo = leg['interchangeTo']
            if interchangeTo and interchangeTo['guaranteed'] == True:
                t += f' guaranteed to {interchangeTo["ToServiceJourney"]["privateCode"]}'

            interchangeFrom = leg['interchangeFrom']
            if interchangeFrom and interchangeFrom['guaranteed'] == True:
                t += f' guaranteed from {interchangeFrom["ToServiceJourney"]["id"]}'

            print(t)


2021-08-24T13:21:29+02:00
	leg foot to "Sandfærhus"
	leg bus to "Siemens"
	leg foot to "Siemens"
2021-08-24T13:38:47+02:00
	leg foot to "Trondheim lufthavn"
	leg bus to "Buran"
	leg bus to "Siemens"
	leg foot to "Siemens"
2021-08-24T13:44:05+02:00
	leg foot to "Sandfærhus"
	leg bus to "Stjørdal stasjon"
	leg bus to "Siemens"
	leg foot to "Siemens"


In [29]:
from time import sleep

# Every unique line from https://github.com/AtB-AS/korrespondanse-check/blob/main/korrespondanser.xlsx
lines = [
    1,2,3,10,14,40,41,42,44,46,70,71,72,73,74,75,76,77,78,79,80,82,83,214,310,311,340,350,410,420,421,422,430,
    440,451,452,455,460,470,480,490,504,505,511,535,541,543,544,552,561,591,592,593,594,595,610,611,612,615,630,
    635,640,660,665,670,680,690,695,722,723,731,733,741,762,763,781,782,783,784,785,5001,5004,5005,5006,5100,
    5101,5102,5103,5104,5106,5110,5111,5205,5206,5207,5209,5210,5211,5212,5300,5305,5400,5401,5402,5404,5406,
    5471,5472,5473,5475,5476,5501,5521,5533,5601,5602,5603,5604,5605,5610,5701,5702,5703,5704,5707,5801,5802,
    5805,5904,7201,7204,7205,7206,7208,7209,7210,7211,7301,7302,7304,7306,7307,7308,7310,7311,7312,7313,7315,
    7317,7318,7322,7323,7327,7328,7329,7330,7341,7402,7403,7404,7405,7408,7410,7411,7412,7414,7415,7416,7417,
    7419,7420,7424,7425,7503,7504,7506,7507,7509,7510,7515,7519,7520,7521,7603,7701,7702,7703,7705,7708,7710,
    7811,7814,7901,7902,7903,7904,7905,7908,7910
]

query = """
query t($line: ID!){
  line(id: $line) {
    id
    name
  }
}
"""

lineIDs = []
missingLines = []

verify_lines = False

for line in lines:
    # Query Entur to se if `ATB:Line:2_{line}` exists
    if verify_lines:
        lineID = f'ATB:Line:2_{line}'
        variables = {
            "line": lineID,
        }
        data = client.execute(query=query, variables=variables)
        try:
            if data['data']['line'] == None:
                print(f'{line}=None')
                missingLines.append(line)
            else:
                print(f'{line}={lineID}')
                lineIDs.append((line,lineID))
        except:
            print(data)
            break

        sleep(1)

    # Assume every `ATB:Line:2_{line}` exists
    else:
        lineID = f'ATB:Line:2_{line}'
        lineIDs.append((line,lineID))
        print(lineID)

ATB:Line:2_1
ATB:Line:2_2
ATB:Line:2_3
ATB:Line:2_10
ATB:Line:2_14
ATB:Line:2_40
ATB:Line:2_41
ATB:Line:2_42
ATB:Line:2_44
ATB:Line:2_46
ATB:Line:2_70
ATB:Line:2_71
ATB:Line:2_72
ATB:Line:2_73
ATB:Line:2_74
ATB:Line:2_75
ATB:Line:2_76
ATB:Line:2_77
ATB:Line:2_78
ATB:Line:2_79
ATB:Line:2_80
ATB:Line:2_82
ATB:Line:2_83
ATB:Line:2_214
ATB:Line:2_310
ATB:Line:2_311
ATB:Line:2_340
ATB:Line:2_350
ATB:Line:2_410
ATB:Line:2_420
ATB:Line:2_421
ATB:Line:2_422
ATB:Line:2_430
ATB:Line:2_440
ATB:Line:2_451
ATB:Line:2_452
ATB:Line:2_455
ATB:Line:2_460
ATB:Line:2_470
ATB:Line:2_480
ATB:Line:2_490
ATB:Line:2_504
ATB:Line:2_505
ATB:Line:2_511
ATB:Line:2_535
ATB:Line:2_541
ATB:Line:2_543
ATB:Line:2_544
ATB:Line:2_552
ATB:Line:2_561
ATB:Line:2_591
ATB:Line:2_592
ATB:Line:2_593
ATB:Line:2_594
ATB:Line:2_595
ATB:Line:2_610
ATB:Line:2_611
ATB:Line:2_612
ATB:Line:2_615
ATB:Line:2_630
ATB:Line:2_635
ATB:Line:2_640
ATB:Line:2_660
ATB:Line:2_665
ATB:Line:2_670
ATB:Line:2_680
ATB:Line:2_690
ATB:Line:2_695
ATB:Li

In [40]:
query = """
query t($line: ID!){
  line(id: $line) {
    id
    name
    quays {
      id
      name
    }

  }
}
"""

#for line in lineIDs:
lineID = "ATB:Line:2_3"
variables = {
    "line": lineID,
}

data = client.execute(query=query, variables=variables)
data['data']['line']['quays'][0]


{'id': 'NSR:Quay:102729', 'name': 'Hallset'}